# Model - Generator hrvatskih tekstova

## 1. INSTALACIJA I UVOZ POTREBNIH BIBLIOTEKA
- PyTorch
- sys, pandaS
- numpy
- re
- os
- io

In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys

sys.stdout.encoding = 'utf-8'

In [25]:
import pandas as pd
import numpy as np
import re
import os
import io
import torch
import matplotlib

## 2. RUKOVANJE TEKSTUALNIM DATOTEKAMA
Zbog pogreške unicodeEncodeError bilo je potrebno uvesti funkciju read_text() koja datoteku za treniranje učitava za odgovarajućim kodiranjem bez izbacivanja pogreške.
Funkcija read_documents_from_directory() očitava sve datoteke unutar direktorija koje se koriste za učenje modela.

In [5]:
common_encodings = [
    'utf-8',
    'utf-16',
    'utf-32',
    'ascii',
    'iso-8859-1',
    'latin-1',
    'cp1252',
    'cp850',
    'cp437',
    'iso-2022-jp',
    'shift_jis',
    'euc-jp',
    'iso-8859-5',
    'koi8-r',
    'cp1251',
    'iso-8859-7',
    'cp1253',
    'iso-8859-15',
    'mac-roman',
    'mac-cyrillic'
]

In [26]:
def read_txt(file_path, common_encodings):
    """Read file content using alternative encodings."""
    for encoding in common_encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as input_file:
                return input_file.read()
        except UnicodeDecodeError:
            continue
    return None

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        combined_text += read_txt(file_path, common_encodings)
    return combined_text

In [27]:
train_directory = './treniranje'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()

In [28]:
with io.open("./treniranje/400_prompt_answer.txt", "w", encoding="utf-8") as f:
    f.write(text_data)

## 3. PRIPREMA ZA DUBOKO UČENJE
Uvoz funkcija iz biblioteke transformers:
- TextDataset
- DataCollatorForLanguageModeling
- GPT2Tokenizer
- GPT2LMHeadModel
- Trainer
- TrainingArguments

Definiranje funkcija load_dataset(), load_data_collator() i train() koji će se koristit u treniranju modela.
Linija torch.cuda.is_available() provjerava dostupnost grafičke kartice u svrhu ubrzavanja procesa treniranja modela.

In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [30]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [31]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [32]:
torch.cuda.is_available()

False

In [33]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [35]:
train_file_path = "./treniranje/400_prompt_answer.txt"
model_name = 'gpt2'
output_dir = 'd:/MODEL/'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 30.0
save_steps = 50000
     

## 4. TRENIRANJE MODELA
Definiranje parametara treniranja. Najbitniji parametar je broj epoha koji određuje koliko će puta podatkovni skup proći kroz model.

In [4]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

NameError: name 'train' is not defined

In [4]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

## 5. TESTIRANJE MODELA
Nakon definiranja funkcija potrebnih za učitavanje modela model je spreman za korištenje i testiranje. Parametar max_len smo ograničili na 124. Time se određuje duljina generiranog odgovora. Maksimalna vrijednost koju je moguće upisati je 1024.

In [5]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [9]:
model_path = "./MODEL"
sequence = "Oči tvoje"
max_len = 124
generate_text(model_path, sequence, max_len)

Oči tvoje ponovo postojiš
ti taj sve sto zato jer smo ljubavi
da samo da si tu
ne može više nisi vatru
neću ti si za tvoj ukrao prije
preko nije mene i odavde bar na tom
Zato jer smo ljubavi
duge to još uvijek svijeta popio
jer smo raste koju suzama
poznat ć
